In [1]:
# // Dependencies
print("Loading Dependencies\n--------------------")

import pandas as pd
print("10%")
import matplotlib
print("20%")
matplotlib.use('TKAgg')
print("30%")
import matplotlib.pyplot as plt
print("40%")
import sys
print("50%")
import os
print("60%")
import tkinter as tk
print("70%")
from tkinter import ttk
print("80%")
from tkinter import messagebox
print("90%")
from tkinter import filedialog
print("100%")
import glob
print("--------------------\nLoaded")

Loading Dependencies
--------------------
10%
20%
30%
40%
50%
60%
70%
80%
90%
100%
--------------------
Loaded


In [2]:
# // Path to Query
#folder_path = 'QUERY_FILE_GOES_HERE/'

#xlsx_files = glob.glob(os.path.join(folder_path, '*.xlsx'))

#QUERYS = {}
#Q_Index = 0

#for file_path in xlsx_files:
#    OPName = os.path.splitext(os.path.basename(file_path))[0] # Extract filename without extension as DataFrame name
#    QUERYS[OPName] = pd.read_excel(file_path)
#    Q_Index += 1
    
#print(f"Files in folder '{folder_path}':")

#if Q_Index > 0:
#    for file_name in QUERYS.keys():
#        print(file_name)
#else:
#   print(f"No .xlsx file found in {folder_path}")

In [3]:
root = tk.Tk()
Query_Path = filedialog.askopenfilename(initialdir='QUERY_FILE_GOES_HERE/', title='Select a file', filetypes=[('Excel files', '*.xlsx')])
Query = pd.read_excel(Query_Path)
terms = []
enroll_status_cols = Query.filter(like='_Enroll_Status').columns
for term in enroll_status_cols:
    term = term[:6]
    terms.append(term)
    print(term)

202230
202310
202320


In [4]:
root.destroy()
root = tk.Tk()
root.title("Current Term Input")

label = tk.Label(root, text="Input current term in Banner format: \nOptions are:")
label.pack()
label = tk.Label(root, text="\n".join(terms))
label.pack()

entry = tk.Entry(root)
entry.pack()

def get_term():
    global Current_Term
    global LastTwo
    Current_Term = entry.get()
    if Current_Term.isdigit():
        Current_Term = int(Current_Term)
        LastTwo = Current_Term % 100
        if Current_Term > 100000 and Current_Term < 999999 and (LastTwo == 10 or LastTwo == 20 or LastTwo == 30):
            root.destroy()
        else:
            messagebox.showerror("Error", "Input not in correct Banner format")
    else:
        messagebox.showerror("Error", "Input is not a number")

submit_btn = tk.Button(root, text="Submit", command=get_term)
submit_btn.pack()

root.mainloop()
print(f"Term selected is " + str(Current_Term))

Term selected is 202310


In [5]:
# Returns Count of VIDs as Population
TPS = Query["VID"].count()

# Sets Term Codes & Reads Current Term from input
if LastTwo == 10:
    Fall = Current_Term
    Term_Actual = "Fall"
elif LastTwo == 20:
    Fall = Current_Term - 10
    Term_Actual = "Spring"
elif LastTwo == 30:
    Fall = Current_Term - 20
    Term_Actual = "Summer"
    Spring = Fall + 10
    Summer = Spring + 10

# Returns Count of Enrolled OP Students in Current Term
CTerm_Search = str(Current_Term) + "_Enroll_Status"
CTerm = Query[CTerm_Search].count()

# Returns Count of Enrolled OP Students in NEXT Term
# Sets Next Term Variable
if LastTwo == 30:
    NTerm_Search = str(Spring) + "_Enroll_Status"
elif LastTwo == 10 or LastTwo == 20:
    NTerm_Search = str(Current_Term + 10) + "_Enroll_Status"
NTerm = Query[NTerm_Search].count()

# 1. Filters out OP Students NOT ENROLLED in current Term
# 2. Returns count of Current Term Enrollees that Enrolled in the Following Term
# 3. Returns count of Current Term withdrawls
Current_E_List = Query[Query[CTerm_Search].notnull()]
Next_E_List_Count = Current_E_List[NTerm_Search].count()
Current_WD = len(Current_E_List[Current_E_List[CTerm_Search] == "WT"])

# 1. Filters OP Students Admitted Next Term or later
# 2. Out of filtered OP Students, counts enrolled students in the next term
DA_NTerm_List = Query[Query["TermCodeAdmit"] > Current_Term]
Admit_DA_NTerm = DA_NTerm_List[NTerm_Search].count()

# Index for graphs: count of all students admitted during or after next term
DASIndex = len(Query[Query["TermCodeAdmit"] > Current_Term].count())

In [6]:
Eratio = Next_E_List_Count / CTerm

labels = ['Enrolled', 'Not Enrolled']
sizes = [Eratio, 1-Eratio]

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, startangle=90, counterclock=False, autopct='%1.1f%%')

plt.title('Students Who Enrolled in Current Term \n That Enrolled in Next Term')

plt.show()

In [7]:
WDRatio = Current_WD / CTerm

labels = ['Withdrawn', 'Enrolled']
sizes = [WDRatio, 1-WDRatio]

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, startangle=90, counterclock=False, autopct='%1.1f%%')

plt.title('Students Enrolled \n Current Term \n That Withdrew')

plt.show()

In [8]:
DAS_Enroll = Admit_DA_NTerm / DASIndex

labels = ['Enrolled', 'Not Yet Enrolled']
sizes = [DAS_Enroll, 1-DAS_Enroll]

fig, ax = plt.subplots()
ax.pie(sizes, labels=labels, startangle=90, counterclock=False, autopct='%1.1f%%')

plt.title('Enrollment of Students Admitted \n Next Term Code or After')

plt.show()

In [9]:
with open("report.txt", "w") as file:
    file.write("Osceola Prosper Report")
    file.write("\n ------------------------------------------")
    file.write("\nTotal Prosper Students: " + str(TPS))
    file.write("\nFall Enrolled Students: " + str(CTerm))
    file.write("\nSpring Enrolled Students: " + str(NTerm))
    file.write("\nFall Students Who Withdrew from Classes: " + str(Current_WD) + " | " + str(format((WDRatio * 100), '.2f')) +"% of Fall Students Withdrew.")
    file.write("\nStudents Admitted in 202320 or later who are Enrolled: " + str(Admit_DA_NTerm) + " | " + str(format((DAS_Enroll * 100), '.2f')) + "% of Spring Admitted Students are Enrolled.")